In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spam-text-message-classification/SPAM text message 20170820 - Data.csv


In [2]:
df = pd.read_csv('../input/spam-text-message-classification/SPAM text message 20170820 - Data.csv')

In [3]:
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  5572 non-null   object
 1   Message   5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [5]:
df.shape

(5572, 2)

In [6]:
df.isnull().sum()

Category    0
Message     0
dtype: int64

# Data Cleaning 

In [7]:
import re 
import nltk 
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer

In [8]:
ps = PorterStemmer()

In [9]:
#lowercasing the message 
# df['txt_clean'] = df['Message'].apply(lambda x:x.lower())

In [10]:
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


#

In [11]:
corpus = []
for i in range(len(df)):
    clean_txt = re.sub('[^a-zA-z]',' ',df['Message'][i])
    clean_txt = clean_txt.lower()
    clean_txt = nltk.word_tokenize(clean_txt)
    clean_txt = [ps.stem(word) for word in clean_txt if word not in set(stopwords.words('english'))]
    clean_txt = ' '.join(clean_txt)
    corpus.append(clean_txt)
    

In [12]:
#remove numerics and symblos 
# def remove_chars(txt):
#     clean_txt = re.compile('[^a-zA-Z]')
#     return clean_txt.sub(' ',txt)

# df['txt_clean'] = df['txt_clean'].apply(remove_chars)

# Extract Feature text 

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000)


In [14]:
X = cv.fit(corpus)

In [15]:
print(X.get_feature_names())

['____', 'aa', 'aah', 'aaniy', 'aaooooright', 'aathi', 'ab', 'abbey', 'abdomen', 'abeg', 'abel', 'aberdeen', 'abi', 'abil', 'abiola', 'abj', 'abl', 'abnorm', 'abouta', 'abroad', 'absenc', 'absolut', 'absolutli', 'abstract', 'abt', 'abta', 'aburo', 'abus', 'ac', 'academ', 'acc', 'accent', 'accentur', 'accept', 'access', 'accid', 'accident', 'accommod', 'accommodationvouch', 'accomod', 'accordin', 'accordingli', 'account', 'accumul', 'ach', 'achan', 'achiev', 'acid', 'acknowledg', 'acl', 'acnt', 'aco', 'across', 'act', 'actin', 'action', 'activ', 'actor', 'actual', 'ad', 'adam', 'add', 'addamsfa', 'addi', 'addict', 'address', 'adewal', 'adi', 'adjust', 'admin', 'administr', 'admir', 'admiss', 'admit', 'ador', 'adp', 'adress', 'adrian', 'adrink', 'adsens', 'adult', 'advanc', 'adventur', 'advic', 'advis', 'advisor', 'ae', 'aeronaut', 'aeroplan', 'afew', 'affair', 'affect', 'affection', 'affidavit', 'afford', 'afghanistan', 'afraid', 'africa', 'african', 'aft', 'afternon', 'afternoon', 'aft

/opt/conda/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [16]:
X_transformed = X.transform(corpus).toarray()

In [17]:
#create data frame with these features 
# df_clean = pd.DataFrame(X_transformed,columns=X.get_feature_names()).add_prefix('Counts_')


In [18]:
# df_new=pd.concat([df,df_clean],axis=1, sort=False)


In [19]:
# df_new.head()

In [20]:
df['Category'] = df['Category'].replace({'spam':0,'ham':1})
df.head()

,Category,Message
0,1,"Go until jurong point, crazy.. Available only ..."
1,1,Ok lar... Joking wif u oni...
2,0,Free entry in 2 a wkly comp to win FA Cup fina...
3,1,U dun say so early hor... U c already then say...
4,1,"Nah I don't think he goes to usf, he lives aro..."


In [21]:
y = df.iloc[:, 0]

In [22]:
y.value_counts()


1    4825
0     747
Name: Category, dtype: int64

# Model

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB



In [24]:
X_train, X_test, y_train, y_test = train_test_split(X_transformed,y,test_size=0.2,stratify=y)

In [25]:
model = MultinomialNB()

In [26]:
spam_detect_model = model.fit(X_train,y_train)

In [27]:
predictions = spam_detect_model.predict(X_test)

In [28]:
a = {'Actual':y_test,'predict':predictions}

In [29]:
pd.DataFrame(a).tail(50)

,Actual,predict
2394,1,1
1233,1,1
605,1,1
520,1,1
1547,1,1
4050,1,1
4577,0,0
5044,1,1
229,1,1
4936,1,1


# Evaluation 

In [30]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from scikitplot.metrics import plot_confusion_matrix, plot_roc

In [31]:
confusion_matrix(y_test, predictions)

array([[141,   8],
       [ 15, 951]])

In [32]:
accuracy_score(y_test, predictions)

0.979372197309417

In [33]:
 print(f'Precision Score of the model: {precision_score(y_test,predictions)}')

Precision Score of the model: 0.9916579770594369


In [34]:
 print(f'Recall Score of the model: {recall_score(y_test,predictions)}')

Recall Score of the model: 0.984472049689441
